# Construcción del dataset de calificaciones

Este notebook documenta el proceso de transformación de las bases de datos de calificaciones de los años **2021 y 2022** en un formato unificado, estructurado y anonimizado.  

Fusagasugá: 2021 - 2022  
Girardot: 2022

In [1]:
import pandas as pd
import os
import sys


current_dir = os.path.dirname(os.path.abspath('__file__'))
project_root = os.path.abspath(os.path.join(current_dir, '..', '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from scripts.utils.hash_utility import HashUtility

In [2]:
archivo_excel = '../../data/raw/calificaciones/excel/calificaciones_2021_2023.xlsx'
df_asignaturas = pd.read_excel(archivo_excel, sheet_name="Asignaturas")
df_bd = pd.read_excel(archivo_excel, sheet_name="BD")

In [3]:
df_bd["Identificación"] = df_bd["Identificación"].astype(str).str.strip()

## Anonimización de los datos

Para proteger la identidad de los estudiantes, se aplicó **hashing criptográfico estable** a las columnas:

- `Identificación`
- `Estudiante` (nombre completo)

Esto permite mantener la trazabilidad interna sin comprometer la privacidad de los estudiantes.

In [4]:
df_bd["Identificación"] = df_bd["Identificación"].apply(HashUtility.hash_stable)
df_bd["Estudiante"] = df_bd["Estudiante"].apply(HashUtility.hash_stable)

In [5]:
df_bd.head()

,Sede,Año,Periodo,Grado,Identificación,Estudiante,Cog_CN,Proc_CN,Axi_CN,Act_CN,...,Axi_EA,Act_EA,Com_EA,Prom_EA,Cog_CL,Proc_CL,Axi_CL,Act_CL,Com_CL,Prom_CL
0,Fusagasugá,2021,1,1,b0d1ca4c178f43fc9a7249f1dcc4f3e41b5775dc08b503...,4be9e7c5911bcec1395515f1529fb5771ea9dd18875296...,80.0,90.0,79.0,78.0,...,100.0,100.0,100.0,99.4,99.0,100.0,100.0,100.0,100.0,99.8
1,Fusagasugá,2021,1,1,27a694edda081c611dca2b98fbf5cff1e215c900499613...,857a5cab0787d8c455c6a279f33ef8a762703bafaa5bd0...,80.0,80.0,80.0,80.0,...,99.0,99.0,76.0,87.2,86.0,85.0,100.0,100.0,100.0,94.2
2,Fusagasugá,2021,1,1,61a4b57fc5308dcfeb2714bc65604400aa4c4591eebb81...,7762d974ea6058c492404ac53d4c54c26d9bdd84db4c0d...,85.0,80.0,90.0,85.0,...,100.0,100.0,81.0,89.6,87.0,97.0,100.0,100.0,100.0,96.8
3,Fusagasugá,2021,1,1,31e68e369bd4094d6cafc97f283caf7afa4b4dcb05a88c...,cbf4bb9d9b3385050519a4997abdf1ca5b066201849824...,85.0,80.0,90.0,85.0,...,98.0,98.0,74.0,84.6,80.0,80.0,88.0,100.0,100.0,89.6
4,Fusagasugá,2021,1,1,79d87fd0646c96a46219ece6f4e7aa4fb9f5fae261e47a...,553d14d4a9643f6f0f9763f2895c2837430a2045a7a845...,85.0,89.0,90.0,85.0,...,100.0,100.0,91.0,93.8,80.0,84.0,100.0,100.0,100.0,92.8


## Comprensión de la Estructura Original

Cada fila de la hoja `BD` contiene múltiples calificaciones para un mismo estudiante y periodo, distribuidas en columnas codificadas por dimensión (ej. `Cog_CS`, `Proc_MT`) donde:

- El prefijo indica la **dimensión evaluativa**:
  - `Cog`: Cognitiva
  - `Proc`: Procedimental
  - `Axi`: Axiológica
  - `Act`: Actitudinal
  - `Com`: Comunicativa
  - `Prom`: Promedio de la asignatura

- El sufijo corresponde a la **abreviatura de la asignatura**:
  - `CN`, `CS`, `MT`, `LC`, `IN`, `IE`, `EF`, `EA`, `CL`, etc.

Mapeo de aisgnaturas

In [6]:
abv_to_nombre = dict(zip(df_asignaturas["Abv"], df_asignaturas["Asignatura"]))
abv_to_nombre

{'CN': 'Ciencias Naturales',
 'CS': 'Ciencias Sociales',
 'MT': 'Matemáticas',
 'LC': 'Lengua Castellana',
 'IN': 'Inglés',
 'IE': 'Innovación y Emprendimiento',
 'EF': 'Educación Física',
 'EA': 'Educación Ambiental',
 'CL': 'Comprensión Lectora'}

## Transformación de Formato Wide a Long

Convertimos la estructura wide en **formato long**, donde cada fila representa una calificación por estudiante, asignatura y dimensión evaluativa:

- Se identificaron todas las asignaturas presentes mediante análisis de sufijos.
- Se mapeó cada abreviatura (`abv`) a su nombre completo usando el diccionario de asignaturas.
- Se generaron registros combinando:
  - Datos base del estudiante (`Sede`, `Año`, `Periodo`, `Grado`, `ID`, `Estudiante`)
  - Nombre de la asignatura
  - Valores por dimensión (`Cog`, `Proc`, `Axi`, `Act`, `Com`, `Prom`)

In [7]:
competencias = ["Cog", "Proc", "Axi", "Act", "Com", "Prom"]
datos_generales = ["Sede", "Año", "Periodo", "Grado", "Identificación", "Estudiante"]

# Encontrar todas las abreviaturas usadas en las columnas
asignaturas_abv = sorted(set(col.split('_')[-1] for col in df_bd.columns if any(col.startswith(c + "_") for c in competencias)))

registros = []

for _, fila in df_bd.iterrows():
    datos_base = {col: fila[col] for col in datos_generales}
    for abv in asignaturas_abv:
        asignatura_nombre = abv_to_nombre.get(abv, abv)
        registro = {
            **datos_base,
            "Asignatura": asignatura_nombre
        }
        for comp in competencias:
            col_name = f"{comp}_{abv}"
            registro[comp] = fila.get(col_name)
        registros.append(registro)

# Convertir a DataFrame
df_final = pd.DataFrame(registros)

## Estructura Final del Dataset


Columnas del dataset transformado:

- `Sede`, `Año`, `Periodo`, `Grado`, `Identificación`, `Estudiante`
- `Asignatura`
- `Cog`, `Proc`, `Axi`, `Act`, `Com`, `Prom`

Cada fila representa una evaluación de una asignatura específica por estudiante en un periodo dado, lo que permite análisis más finos como:

- Seguimiento del desempeño por competencia
- Comparación interasignaturas e intersedes
- Construcción de indicadores agregados por estudiante

In [8]:
df_final

,Sede,Año,Periodo,Grado,Identificación,Estudiante,Asignatura,Cog,Proc,Axi,Act,Com,Prom
0,Fusagasugá,2021,1,1,b0d1ca4c178f43fc9a7249f1dcc4f3e41b5775dc08b503...,4be9e7c5911bcec1395515f1529fb5771ea9dd18875296...,Comprensión Lectora,99.0,100.0,100.0,100.0,100.0,99.8
1,Fusagasugá,2021,1,1,b0d1ca4c178f43fc9a7249f1dcc4f3e41b5775dc08b503...,4be9e7c5911bcec1395515f1529fb5771ea9dd18875296...,Ciencias Naturales,80.0,90.0,79.0,78.0,88.0,83.0
2,Fusagasugá,2021,1,1,b0d1ca4c178f43fc9a7249f1dcc4f3e41b5775dc08b503...,4be9e7c5911bcec1395515f1529fb5771ea9dd18875296...,Ciencias Sociales,84.0,91.0,89.0,89.0,100.0,90.6
3,Fusagasugá,2021,1,1,b0d1ca4c178f43fc9a7249f1dcc4f3e41b5775dc08b503...,4be9e7c5911bcec1395515f1529fb5771ea9dd18875296...,Educación Ambiental,98.0,99.0,100.0,100.0,100.0,99.4
4,Fusagasugá,2021,1,1,b0d1ca4c178f43fc9a7249f1dcc4f3e41b5775dc08b503...,4be9e7c5911bcec1395515f1529fb5771ea9dd18875296...,Educación Física,94.0,94.0,95.0,95.0,100.0,95.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16618,Girardot,2022,4,6,043cd6f2a902dda0eb029614ee22c66443d5b60ef92a39...,68d8d19e058e844aee87ff0312324452a1ac6326928a28...,Educación Física,90.0,90.0,92.0,89.0,89.0,90.0
16619,Girardot,2022,4,6,043cd6f2a902dda0eb029614ee22c66443d5b60ef92a39...,68d8d19e058e844aee87ff0312324452a1ac6326928a28...,Innovación y Emprendimiento,89.0,92.0,96.0,94.0,89.0,92.0
16620,Girardot,2022,4,6,043cd6f2a902dda0eb029614ee22c66443d5b60ef92a39...,68d8d19e058e844aee87ff0312324452a1ac6326928a28...,Inglés,76.0,74.0,82.0,74.0,70.0,75.2
16621,Girardot,2022,4,6,043cd6f2a902dda0eb029614ee22c66443d5b60ef92a39...,68d8d19e058e844aee87ff0312324452a1ac6326928a28...,Lengua Castellana,82.0,83.0,90.0,88.0,77.0,84.0


## Guardado del Dataset Procesado

In [9]:
# Guardar el DataFrame en un archivo CSV
output_path = os.path.join(project_root, 'data', 'interim', 'calificaciones',  'calificaciones_2021_2023.csv')
df_final.to_csv(output_path, index=False)